<a href="https://colab.research.google.com/github/seanreed1111/colab-demos/blob/master/DORA_gpt_3_5_turbo_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:32022R2554


In [1]:
!mkdir -p DATA/DORA;curl https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:32022R2554 -o DATA/DORA/CELEX_32022R2554.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1458k    0 1458k    0     0  3136k      0 --:--:-- --:--:-- --:--:-- 3136k


In [2]:
%pip install -qq llama-index openai pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.0/584.0 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.8/254.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.5 MB/s eta 0:00:00


In [3]:
import llama_index
from llama_index import (
    ListIndex, VectorStoreIndex, SimpleDirectoryReader, LLMPredictor,
    ServiceContext,StorageContext, load_index_from_storage
)
from llama_index.response.notebook_utils import display_response
from llama_index.llms import OpenAI
from IPython.display import Markdown, display
import logging, sys, os, json
from pathlib import Path
import openai
from typing import List, Tuple, Dict, Union

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger()
logger.addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
cwd: Path = Path.cwd()
data_dir: Path = cwd / "DATA"
dora_dir: Path  = data_dir / "DORA"
documents: List = SimpleDirectoryReader(dora_dir).load_data()

In [5]:
def set_open_ai_key(env_path: Union[Path, str] = None) -> Tuple[bool, str]:
  import json, os
  from pathlib import Path

  openai.api_key = None #clear previous key if exists
  env_path: Path = Path(env_path).absolute() # in case env_path is passed as a str
  if not env_path.is_file(): err: str = f"File:{str(env_path)} does not exist."
  else:
    try:
      with open(env_path, "r") as f:
          env_vars = json.load(f)
      os.environ["OPENAI_API_KEY"] = env_vars["OPENAI_API_KEY"]
      openai.api_key = os.environ["OPENAI_API_KEY"]
      openai.Model.list() #test a random command on the openai API
      err = None
    except Exception as e:
      err = json.dumps({"error":e})

  logger.info(err)
  return (True, "ok") if not err else (False, err)

In [6]:
cwd: Path = Path.cwd()
env_path: Path = cwd/ "openai.env";
print(set_open_ai_key(env_path)) # Tuple[bool, str]

(True, 'ok')


In [8]:
TEMPERATURE = 0

llm: llama_index.llms.openai.OpenAI = OpenAI(
    temperature=TEMPERATURE,
    model="gpt-3.5-turbo",
    max_tokens=500,
    max_retries = 5)

service_context: llama_index.indices.service_context.ServiceContext = (
    ServiceContext.from_defaults(llm=llm)
    )

In [9]:
index: llama_index.indices.vector_store.base.VectorStoreIndex = (
     VectorStoreIndex.from_documents(documents, service_context=service_context)
)
index.storage_context.persist(persist_dir="./storage")

In [ ]:
# store and reload index from memory
# from llama_index import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage_context)

In [19]:
import pandas as pd
sample_qna_path = Path("/content/DATA/SampleQuestionsAndAnswers/Questionnaire for Chat PDF - example DORA_v0.1.xlsx")
df = pd.read_excel(sample_qna_path, skiprows=1);df.head()

,Serial,Questions,Answers,Category,Comments,Feedback
0,1,What are the key summary points of this regula...,o\tAs an expert on the content of this PDF doc...,General,NaN,NaN
1,2,Explain the regulation to me in brief,Regulation (EU) 2022/2554 is a legislative act...,General,NaN,NaN
2,3,How is this regulation expected to impact a gl...,The regulation aims to establish a common fram...,General,NaN,NaN
3,4,What changes do financial institutions have to...,"o\tAs an AI language model, I do not have acce...",General,NaN,NaN
4,5,•\tWhat is the scope of this regulation? Does ...,"o\tYes, the regulation in this PDF has a speci...",General,NaN,NaN


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Serial     38 non-null     int64  
 1   Questions  38 non-null     object 
 2   Answers    38 non-null     object 
 3   Category   38 non-null     object 
 4   Comments   2 non-null      object 
 5   Feedback   0 non-null      float64
dtypes: float64(1), int64(1), object(4)
memory usage: 1.9+ KB


In [24]:
QA = list(zip(df['Questions'].tolist(), df['Answers'].tolist()))
QA[0]

('What are the key summary points of this regulation as per this PDF document?',
 'o\tAs an expert on the content of this PDF document, I can provide you with a summary of the key points of this regulation: - The aim of this regulation is to ensure a high level of digital operational resilience for regulated financial entities in the EU. - Financial entities are required to identify and manage their ICT-related risks, and to establish and maintain effective ICT risk management frameworks. - Financial entities must report major ICT-related incidents to the relevant competent authority, and may also voluntarily report significant cyber threats. - The regulation establishes a framework for cooperation and information-sharing between competent authorities, financial entities, and other relevant stakeholders. - Small and non-interconnected investment firms, payment institutions, electronic money institutions, and small institutions for occupational retirement provision are exempted from cer

In [21]:
query_engine: llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine = (
    index.as_query_engine()
)
q: str = 'What are the key summary points of this regulation as per this PDF document?'
response: llama_index.response.schema.Response = query_engine.query(q)
display_response(response)

**`Final Response:`** The key summary points of this regulation, as per the PDF document, are as follows:

- The objective of this regulation is to achieve a high level of digital operational resilience for regulated financial entities.
- The regulation lays down uniform requirements for the security of network and information systems supporting the business processes of financial entities.
- It includes requirements for ICT risk management, reporting of major ICT-related incidents, digital operational resilience testing, information sharing on cyber threats and vulnerabilities, and measures for the management of ICT third-party risk.
- The regulation also covers contractual arrangements between ICT third-party service providers and financial entities, the establishment and conduct of the Oversight Framework for critical ICT third-party service providers, and cooperation among competent authorities.
- It is considered a sector-specific Union legal act for essential or important financial entities.
- The regulation does not affect Member States' responsibility for essential State functions related to public security, defense, and national security.
- The European Data Protection Supervisor was consulted and provided an opinion on the regulation.
- The regulation aims to consolidate and upgrade ICT risk requirements as part of operational risk requirements, addressing components of operational resilience and ICT-related incidents.
- It builds upon existing Union legal acts that covered various financial risk categories but did not comprehensively address operational resilience.

In [25]:
for q,a in QA[1:2]:
  print("\n","\n", q, "\n")
  display_response(query_engine.query(q))


 
 Explain the regulation to me in brief 



**`Final Response:`** The regulation aims to achieve a high level of digital operational resilience for regulated financial entities. It establishes uniform requirements for the security of network and information systems supporting the business processes of financial entities. These requirements include ICT risk management, reporting of major incidents and cyber threats, digital operational resilience testing, information sharing on cyber threats and vulnerabilities, and measures for managing third-party ICT risks. The regulation also covers contractual arrangements between ICT service providers and financial entities, the establishment and conduct of the Oversight Framework for critical ICT service providers, cooperation among competent authorities, and supervision and enforcement. It is applicable to financial entities identified as essential or important entities and does not affect Member States' responsibilities regarding public security, defense, and national security.

In [26]:
for q,a in QA[2:3]:
  print("\n","\n", q, "\n", "\n")
  display_response(query_engine.query(q))


 
 How is this regulation expected to impact a global financial institution with operations in north america, europe and asia 
 



**`Final Response:`** Based on the provided context information, this regulation is expected to impact a global financial institution with operations in North America, Europe, and Asia in the following ways:

1. Strengthening Digital Operational Resilience: The regulation aims to strengthen the digital operational resilience of the financial services industry. This means that the global financial institution will be required to enhance its ability to withstand and recover from ICT breaches and incidents, ensuring the safety and good functioning of its technological infrastructure.

2. Harmonization of ICT Risk Provisions: The regulation seeks to address legislative disparities and uneven national regulatory or supervisory approaches to ICT risk. This harmonization will help remove obstacles to the functioning of the internal market in financial services, facilitating the smooth exercise of the freedom of establishment and provision of services for financial entities operating on a cross-border basis. This will benefit the global financial institution by promoting consistency and reducing regulatory burdens across different jurisdictions.

3. Compliance with Key Digital Operational Resilience Requirements: The regulation calls for further harmonization of key digital operational resilience requirements for all financial entities. This means that the global financial institution will need to comply with these requirements to develop its ICT capabilities and overall resilience, ensuring it can withstand operational outages. Compliance with these requirements will contribute to the stability and integrity of the Union financial markets, protecting investors and consumers. 

4. Consistency with International Standards: The regulation aligns with international efforts to enhance digital resilience and coordinate regulatory or supervisory work. This means that the global financial institution, with operations in North America, Europe, and Asia, will need to ensure its compliance with relevant international standards and best practices to bolster the resilience of its financial systems.

Overall, this regulation is expected to have a significant impact on the global financial institution, requiring it to strengthen its digital operational resilience, comply with harmonized requirements, and align with international standards. It aims to enhance the stability and integrity of the financial sector, which will benefit the institution's operations in North America, Europe, and Asia by promoting consistency, reducing regulatory obstacles, and protecting investors and consumers.

In [27]:
for q,a in QA[3:4]:
  print("\n","\n", q, "\n", "\n")
  display_response(query_engine.query(q))


 
 What changes do financial institutions have to make in order to comply with this regulation in this PDF? 
 



**`Final Response:`** Financial institutions have to maintain updated ICT systems that are reliable and capable of guaranteeing the processing of data required for their services. They also need to ensure sufficient technological resilience to deal adequately with additional processing needs due to stressed market conditions or other adverse situations. Additionally, payment institutions, account information service providers, and electronic money institutions are required to report all operational or security payment-related incidents, regardless of the ICT nature of the incident.

In [29]:
q = "What is the scope of this regulation as defined in article 2 of page 24 of this document ?"
display_response(query_engine.query(q))

**`Final Response:`** The scope of this regulation, as defined in Article 2 of page 24 of this document, includes various entities such as credit institutions, payment institutions, account information service providers, electronic money institutions, investment firms, crypto-asset service providers, central securities depositories, trading venues, insurance and reinsurance undertakings, and many others. These entities are collectively referred to as "financial entities" in this regulation. However, there are certain exemptions mentioned in Article 2(3) of the regulation, such as managers of alternative investment funds, insurance and reinsurance undertakings, and institutions for occupational retirement provision with fewer than 15 members.

In [32]:
QA[29]

('•\tWhat is the scope of this regulation as defined in article 2 of page 24 of tis document ?',
 'o\tArticle 2 of page 24 of the PDF document defines the scope of the regulation. The regulation applies to the following entities: 1. Financial entities, including credit institutions, payment institutions, electronic money institutions, and account information service providers, as well as market operators, investment firms, and other entities that are subject to authorization or registration under EU financial services legislation. 2. ICT service providers that provide services to financial entities referred to in point 1 above. 3. Competent authorities responsible for the supervision of financial entities referred to in point 1 above, as well as ICT service providers referred to in point 2 above. The regulation applies to the entities referred to above when they operate in the EU, regardless of whether they are established in the EU or in a third country. The regulation does not apply 

In [33]:
for q,a in QA[19:20]:
  print("\n","\n", q, "\n", "\n")
  display_response(query_engine.query(q))


 
 I am an IT analyst, will I be impacted by this regulation? 
 



**`Final Response:`** Based on the provided context information, it is not possible to determine whether an IT analyst will be impacted by this regulation. The regulation specifically focuses on critical ICT third-party service providers and their oversight framework. Without further information about the specific role and responsibilities of the IT analyst in relation to critical ICT third-party service providers, it is not possible to determine the impact of this regulation on the IT analyst.

In [34]:
for q,a in QA[26:27]:
  print("\n","\n", q, "\n", "\n")
  display_response(query_engine.query(q))


 
 What governance structure is needed at an organisation to comply with this regulation? 
 



**`Final Response:`** Based on the provided context information, it is not possible to determine the specific governance structure needed at an organization to comply with this regulation. The context information mentions the need for competent authorities to have supervisory, investigative, and sanctioning powers, as well as the requirement for close cooperation among relevant competent authorities. It also mentions the development of regulatory technical standards by the ESAs (European Supervisory Authorities). However, the specific governance structure required for compliance with this regulation is not explicitly stated.